1) Créer le Semantic Layer 

In [2]:
from sm_model_classes import Semantic_Layer, SemanticModel, Entity , Dimension, Measure
import json 
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec


connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()

SL_JF= self=Semantic_Layer(session, 'analytics','JF_SMS') #Instance de la classe semantic layer

SL_JF.create_semantic_layer() #Créer le Sematic_layer: Schéma dans snowflake 

 create schema if not exists analytics.JF_SMS 
 [Row(status='JF_SMS already exists, statement succeeded.')] 

 create tag if not exists analytics.JF_SMS.SNOW_MS_JAFFLE_SHOP_OBJECT 
 [Row(status='SNOW_MS_JAFFLE_SHOP_OBJECT already exists, statement succeeded.')] 

 create tag if not exists analytics.JF_SMS.SNOW_MS_JAFFLE_SHOP_SEMANTIC_MODEL_METADATA 
 [Row(status='SNOW_MS_JAFFLE_SHOP_SEMANTIC_MODEL_METADATA already exists, statement succeeded.')] 

 create tag if not exists analytics.JF_SMS.SNOW_MS_JAFFLE_SHOP_SEMANTIC_MODEL_DIMENSIONS 
 [Row(status='SNOW_MS_JAFFLE_SHOP_SEMANTIC_MODEL_DIMENSIONS already exists, statement succeeded.')] 

 create tag if not exists analytics.JF_SMS.SNOW_MS_JAFFLE_SHOP_MEASURES 
 [Row(status='SNOW_MS_JAFFLE_SHOP_MEASURES already exists, statement succeeded.')] 



2) Définition des Entities

In [3]:
e = Entity(entity_name='ORDERS', join_keys='order_id',desc='Primary Key for orders')   #entité "Orders" définie par la clé order_id
SL_JF.register_entity(e)    #enregistrer l'entité orders dans le semantic layer SL_JF


CREATE TAG IF NOT EXISTS analytics.JF_SMS.SNOW_METRIC_STORE_ENTITY_ORDERS
                    ALLOWED_VALUES 'order_id'
                    COMMENT = 'Primary Key for orders' 
 [Row(status='SNOW_METRIC_STORE_ENTITY_ORDERS already exists, statement succeeded.')] 



In [4]:
SL_JF.get_entities()   #lister les entités du semantic layer
#penser à ajouterle user & date de mise à jour 


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"                           |"database_name"  |"schema_name"  |"owner"       |"comment"               |"allowed_values"  |"owner_role_type"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-07-14 11:26:47.928000-07:00  |SNOW_METRIC_STORE_ENTITY_ORDERS  |ANALYTICS        |JF_SMS         |ACCOUNTADMIN  |Primary Key for orders  |["order_id"]      |ROLE               |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------
|"E

Ajouter des modeles au SL: 

In [5]:
from sm_model_classes import Semantic_Layer, SemanticModel, Entity , Dimension, Measure


SM=SemanticModel('orders','RAW','DMT_SCHEMA','FCT_ORDERS',e,'order_date','Order fact table. This table’s grain is one row per order')


SL_JF.register_semantic_model(SM)

CREATE or replace view analytics.JF_SMS.orders
                COMMENT = ' '
                TAG (
                    analytics.JF_SMS.SNOW_MS_JAFFLE_SHOP_OBJECT = '{"type": "sm_view"}',
analytics.JF_SMS.SNOW_MS_JAFFLE_SHOP_SEMANTIC_MODEL_METADATA = '{"entities": ["ORDERS"], "timestamp_col": "order_date"}',
analytics.JF_SMS.SNOW_METRIC_STORE_ENTITY_ORDERS = 'order_id'
                )
                AS (select * from RAW.DMT_SCHEMA.FCT_ORDERS) 
 [Row(status='View ORDERS successfully created.')] 



Ajouter les dimensions du Modéle sémantique SM

In [6]:
dimension=Dimension('CUSTOMER_ID','categorical','','the customer id')    #instance
SL_JF.register_dimension(SM,dimension)   #enregistrer la dimension du modèle SM

SL_JF.get_dimensions(SM)      #lister les dimensions du modéle SM

ALTER TABLE analytics.JF_SMS.orders MODIFY COLUMN CUSTOMER_ID SET TAG SNOW_MS_JAFFLE_SHOP_SEMANTIC_MODEL_DIMENSIONS = '{"type":"categorical", "type_params":,"descripton":the customer id}';
 
 [Row(status='Statement executed successfully.')] 

use schema analytics.JF_SMS;  
 [Row(status='Statement executed successfully.')] 

---------------------------------------------------------------------------------------
|"SEMANTIC_MODEL"  |"DIMENSION"  |"DIMENSION_DETAILS"                                 |
---------------------------------------------------------------------------------------
|ORDERS            |CUSTOMER_ID  |{"type":"categorical", "type_params":,"descript...  |
---------------------------------------------------------------------------------------



In [33]:


measure=Measure('order_total','The total amount for each order including taxes.','sum','amount')#créer une instance mesure 

   
SL_JF.register_measure(SM,measure)     #affecter la mesure measure au SM (colonne utilisée : amount)


SL_JF.get_measures(SM)

use schema analytics.JF_SMS;  
 [Row(status='Statement executed successfully.')] 

---------------------------------------------------------------------------------------------------------------------------------------------
|"SEMANTIC_MODEL"  |"MEASURE"                           |"DESCRIPTION"                                       |"AGGREGATION"  |"EXPRESSION"  |
---------------------------------------------------------------------------------------------------------------------------------------------
|ORDERS            |SNOW_MS_MEASURE_ORDERS_ORDER_TOTAL  |"The total amount for each order including taxes."  |"sum"          |"amount"      |
---------------------------------------------------------------------------------------------------------------------------------------------

